For testing incremental python code

In [2]:
#------------------------
# DEPENDENCIES + SETUP
#------------------------

# dependencies
from flask import Flask, render_template
import pymongo as pym
import pandas as pd

# setup mongo connection
conn = 'mongodb://localhost:27017'
client = pym.MongoClient(conn)

# setup mongo variables
db = client.trending_games_db

# setup Flask app
app = Flask(__name__)

#------------------------
# DATA IMPORT
#------------------------
# create list of all the collection names
collection_names = db.list_collection_names()

# dump all collections' data into one mega list
# big_data = [list(db[i].find()) for i in collection_names]

# create individual variables for each collection
steam_data = list(db['steam_charts'].find())
twitch_data = list(db['twitch'].find())
gline_data = list(db['gtrends_multiline'].find())
ggeo_data =  list(db['gtrends_geo'].find())
yahoo_data = list(db['y_finance'].find())

In [5]:
gline_data[0]

{'_id': ObjectId('61e2ff524e1b16fb70170025'),
 'Month': '2004-01',
 'Apex': 0,
 'CSGO': 58,
 'Dota 2': 0,
 'GTA V': 1,
 'Rust': 23}

In [10]:
#timeline information
search_apex = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/multiTimeline(Apex).csv')
search_csgo = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/multiTimeline(CSGO).csv')
search_dota = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/multiTimeline(DOTA).csv')
search_rust = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/multiTimeline(Rust).csv')
search_gta = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/multiTimeline(GTAV).csv')

#geomap information
geo_apex = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/geoMap(Apex).csv')
geo_csgo = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/geoMap(CSGO).csv')
geo_dota = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/geoMap(DOTA).csv')
geo_rust = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/geoMap(Rust).csv')
geo_gta = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/geoMap(GTAV).csv')

#state information
geo_state = pd.read_csv('../Raw/GoogleTrends - Chris/Resources/statelatlong.csv')
geo_state.rename(columns = {'City' : 'Region'}, inplace = True)

#format, clean df multiline
search_ls = [search_apex, search_csgo, search_dota, search_gta, search_rust]
merged_search = search_ls[0].merge(search_ls[1], on = 'Month', how = 'outer')
for i in range(2, len(search_ls)):
    merged_search = merged_search.merge(search_ls[i], on = 'Month', how = 'outer')
merged_search.rename(columns = {merged_search.columns[1] : 'Apex',
                                merged_search.columns[2] : 'CSGO',
                                merged_search.columns[3] : 'Dota 2',
                                merged_search.columns[4] : 'GTA V',
                                merged_search.columns[5] : 'Rust'
}, inplace = True)
merged_search.replace('<1', 1, inplace = True)
merged_search = merged_search.astype({'Apex' : 'int64', 'Dota 2': 'int64'})
search_dict = merged_search.to_dict('records')

#format, clean df geo
geo_ls = [geo_apex, geo_csgo, geo_dota, geo_gta, geo_rust, geo_state]
merged_geo = geo_ls[0].merge(geo_ls[1], on = 'Region', how = 'outer')
for i in range(2, len(geo_ls)):
    merged_geo = merged_geo.merge(geo_ls[i], on = 'Region', how = 'outer')
merged_geo.rename(columns = {merged_geo.columns[1] : 'Apex',
                             merged_geo.columns[2] : 'CSGO',
                             merged_geo.columns[3] : 'Dota 2',
                             merged_geo.columns[4] : 'GTA V',
                             merged_geo.columns[5] : 'Rust'
}, inplace = True)
merged_geo.sort_values('Region', inplace = True)
merged_geo.reset_index(drop = True, inplace = True)
geo_dict = merged_geo.to_dict('records')

In [11]:
geo_dict

[{'Region': 'Alabama',
  'Apex': 66,
  'CSGO': 54,
  'Dota 2': 40,
  'GTA V': 74,
  'Rust': 66,
  'State': 'AL',
  'Latitude': 32.6010112,
  'Longitude': -86.6807365},
 {'Region': 'Alaska',
  'Apex': 72,
  'CSGO': 79,
  'Dota 2': 93,
  'GTA V': 65,
  'Rust': 78,
  'State': 'AK',
  'Latitude': 61.3025006,
  'Longitude': -158.7750198},
 {'Region': 'Arizona',
  'Apex': 82,
  'CSGO': 72,
  'Dota 2': 59,
  'GTA V': 66,
  'Rust': 64,
  'State': 'AZ',
  'Latitude': 34.1682185,
  'Longitude': -111.930907},
 {'Region': 'Arkansas',
  'Apex': 80,
  'CSGO': 53,
  'Dota 2': 45,
  'GTA V': 81,
  'Rust': 77,
  'State': 'AR',
  'Latitude': 34.7519275,
  'Longitude': -92.1313784},
 {'Region': 'California',
  'Apex': 67,
  'CSGO': 95,
  'Dota 2': 79,
  'GTA V': 59,
  'Rust': 62,
  'State': 'CA',
  'Latitude': 37.2718745,
  'Longitude': -119.2704153},
 {'Region': 'Colorado',
  'Apex': 79,
  'CSGO': 59,
  'Dota 2': 54,
  'GTA V': 58,
  'Rust': 67,
  'State': 'CO',
  'Latitude': 38.9979339,
  'Longitude': 

In [5]:
merged_geo
# geo_dict

,Region,Apex,CSGO,Dota 2,GTA V,Rust
0,Alabama,66,54,40,74,66
1,Alaska,72,79,93,65,78
2,Arizona,82,72,59,66,64
3,Arkansas,80,53,45,81,77
4,California,67,95,79,59,62
5,Colorado,79,59,54,58,67
6,Connecticut,62,81,59,53,70
7,Delaware,59,70,55,66,80
8,District of Columbia,13,26,32,32,46
9,Florida,59,66,49,60,68
